In [22]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot


sb.set() # set the default Seaborn style for graphics

In [23]:
print('hellow world')

hellow world


In [24]:
energyData = pd.read_csv('owid-energy-data.csv')
energyData = pd.DataFrame(energyData)
energyData







,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,Afghanistan,1900,AFG,4832414.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1901,AFG,4879685.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1902,AFG,4935122.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1903,AFG,4998861.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1904,AFG,5063419.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21843,Zimbabwe,2017,ZWE,14751101.0,2.194784e+10,NaN,NaN,NaN,NaN,21.693,...,0.136,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21844,Zimbabwe,2018,ZWE,15052191.0,2.271535e+10,NaN,NaN,NaN,NaN,25.910,...,0.218,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21845,Zimbabwe,2019,ZWE,15354606.0,NaN,NaN,NaN,NaN,NaN,24.748,...,0.364,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21846,Zimbabwe,2020,ZWE,15669663.0,NaN,NaN,NaN,NaN,NaN,22.336,...,0.395,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN


In [27]:
data_unclean = pd.read_csv('owid-energy-data.csv')
data_clean = data_unclean.dropna(subset=['country','year','population','gdp','electricity_demand'])
energy = data_clean [['country','year','population','gdp','electricity_demand']]
freq=energy.value_counts(subset='country')
freq.head(50)

country
Malta              29
Italy              29
Spain              29
Slovenia           29
Slovakia           29
North Macedonia    29
Croatia            29
Norway             29
Cyprus             29
Czechia            29
Iceland            29
Denmark            29
Serbia             29
Hungary            29
Estonia            29
Romania            29
Finland            29
France             29
Greece             29
Portugal           29
Poland             29
Sweden             29
Ireland            29
Albania            29
Turkey             29
Luxembourg         29
Lithuania          29
Austria            29
United Kingdom     29
Ukraine            29
Belgium            29
Latvia             29
Germany            29
Bulgaria           29
Netherlands        29
Myanmar            19
Panama             19
Mauritania         19
Mauritius          19
Niger              19
Mexico             19
Peru               19
Nepal              19
Moldova            19
Paraguay           19
Pa

In [25]:

import json
import ipywidgets as widgets
from ipyleaflet import Map, GeoJSON, Choropleth, basemaps, LayerGroup
import branca.colormap as cm



# read the geojson data
with open('custom.geo.json') as f:
    geo_json_data = json.load(f)
    for d in geo_json_data["features"]:
        d["name"] = d["properties"]["name"]
       
# get the min and max year available
min_year = energyData['year'].min()
max_year = energyData['year'].max()
print(min_year)
print(max_year)




1900
2022


In [26]:
# create the base map
m = Map(center=(0, 0), zoom=2, basemap=basemaps.CartoDB.Positron)
# name changer
proper_name_mapping = {
    "Russian Federation":"Russia",
    "Czech Republic":"Czechia",
    "Macedonia, FYR":"Macedonia",
    "Central African Republic":"Central African Rep.",
    "Bosnia and Herzegovina":"Bosnia and Herz.",
    "Slovak Republic":"Slovakia",
    "Iran, Islamic Rep.":"Iran",
    "Somaliland":"Somalia",
    "Dominican Republic":"Dominican Rep.",
    "Venezuela, RB":"Venezuela",
    "Lao PDR":"Laos",
    "Yemen, Rep.":"Yemen",
    "South Sudan":"S. Sudan",
    "Papua New Guinea":"Guinea",
    "Congo, Rep.":"Congo",
    "Congo, Dem. Rep.":"Dem. Rep. Congo",
    "Cyprus, Northern":"N. Cyprus",
    "Kyrgyz Republic":"Kyrgyzstan",
    "Korea, Dem. Rep.":"South Korea",
    "Palestine (West Bank and Gaza)":"Palestinian",
    "Syrian Arab Republic":"Syria",
    "Egypt, Arab Rep.":"Egypt",
    "Gambia, The":"Gambia"

}


# create a colormap
color_scale = cm.LinearColormap(colors=['green', 'yellow', 'red'], vmin=0, vmax=energyData['gdp'].max())

def update_choropleth(year):
    # filter the data for the current year and remove the rows where the GDP value is missing
    
    try:
        choro_data = energyData.loc[(energyData['year'] == year) & (energyData['gdp'].notna()),].copy()
    except KeyError:
        # If a KeyError is raised, add a new row with 'gdp' equal to 0 for each missing country value
        missing_countries = set(energyData['country']) - set(choro_data['country'])
        new_rows = [{'year': year, 'country': country, 'gdp': 0} for country in missing_countries]
        new_df = pd.DataFrame(new_rows)
        choro_data = choro_data.append(new_df, ignore_index=True)

    
        
    choro_data  = dict(zip(choro_data["country"].tolist(), choro_data["gdp"]))
    for d in geo_json_data["features"]:
        if d["name"] in proper_name_mapping.keys():
            choro_data[d["name"]] = choro_data[proper_name_mapping[d["name"]]]
        elif d["name"] in choro_data:
            choro_data[d["name"]] = choro_data[d["name"]]
        else:
            choro_data[d["name"]] = 0

    print (choro_data)
    # create a choropleth layer
    choropleth_layer = Choropleth(
        geo_data=geo_json_data,
        choro_data=choro_data,
        colormap=color_scale,
        style={'fillOpacity': 1.0, 'weight': 0.5},
        name='GDP Choropleth',
        key_on= "name"
    )
    
    # remove the previous choropleth layer (if any) and add the new one
    layer_group.clear_layers()
    layer_group.add_layer(choropleth_layer)

# create a layer group to hold the choropleth layer
layer_group = LayerGroup()
m.add_layer(layer_group)
# display the map
display(m)

# add the slider to the map

widgets.interact(update_choropleth, year=widgets.IntSlider(min=min_year, max=max_year, step=1, value=min_year))







Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

interactive(children=(IntSlider(value=1900, description='year', max=2022, min=1900), Output()), _dom_classes=(…

<function __main__.update_choropleth(year)>